In [ ]:
#Dostęp do dysku
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Biblioteki
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#Wczytanie dnaych do trenowania
df = pd.read_csv('/content/drive/MyDrive/result.csv', sep='\t')
df = df[['Category','Title','RawWiki','CleanWiki']]
df.head(75)

,Category,Title,RawWiki,CleanWiki
0,Animals,Tiger,The tiger (Panthera tigris) is the largest ext...,the tiger panthera tigris is the largest extan...
1,Animals,Rabbit,Rabbits are small mammals in the family Lepori...,rabbits are small mammals in the family lepori...
2,Animals,Koala Bear,"The koala or, inaccurately, koala bear[a] (Ph...",the koala or inaccurately koala bear a phascol...
3,Animals,Guinea Pig,The guinea pig or domestic guinea pig (Cavia ...,the guinea pig or domestic guinea pig cavia po...
4,Animals,Mouse,"A mouse, plural mice, is a small rodent. Chara...",a mouse plural mice is a small rodent characte...
...,...,...,...,...
70,Countries,Andorra,"Andorra (/ænˈdɔːrə/ (listen), also UK: /ænˈdɒr...",andorra ænˈdɔːrə listen also uk ænˈdɒrə catala...
71,Countries,Zambia,"Zambia (/ˈzæmbiə, ˈzɑːm-/), officially the Rep...",zambia ˈzæmbiə ˈzɑːm officially the republic o...
72,Countries,Vietnam,"Vietnam (also written as Viet Nam, Vietnamese:...",vietnam also written as viet nam vietnamese vi...
73,Countries,Australia,"Australia, officially the Commonwealth of Aust...",australia officially the commonwealth of austr...


In [ ]:
df.shape

(75, 4)

In [ ]:
df['CleanWiki'].apply(lambda x: len(x.split(' '))).sum()

24658

In [ ]:
def tokenize_text(text):
  tokens = []
  for sent in nltk.sent_tokenize(df['CleanWiki'][text]):
  #for sent in nltk.sent_tokenize(text):
    #print(sent)
    for word in nltk.word_tokenize(sent):
      #print(word)
      if len(word) < 2:
        continue
      tokens.append(word.lower())
  #print(tokens)
  return tokens

In [ ]:
print(nltk.sent_tokenize(df['CleanWiki'][1]))

['rabbits are small mammals in the family leporidae along with the hare of the order lagomorpha along with the pika oryctolagus cuniculus includes the european rabbit species and its descendants the world s breeds of domestic rabbit sylvilagus includes wild rabbit species among them the seven types of cottontail the european rabbit which has been introduced on every continent except antarctica is familiar throughout the world as a wild prey animal and as a domesticated form of livestock and pet with its widespread effect on ecologies and cultures the rabbit or bunny is in many areas of the world a part of daily life as food clothing a companion and a source of artistic inspiration although once considered rodents lagomorphs like rabbits have been discovered to have diverged separately and earlier than their rodent cousins and have a number of traits rodents lack like two extra incisors']


In [ ]:
en_stops = set(stopwords.words('english'))
tokens=[]
for i in range(len(df['CleanWiki'])):
  toke = tokenize_text(i)
  #print(toke)
  one =[]
  for x in toke:
    if x not in en_stops:
      one.append(x)
  tokens.append(one)

print(tokens)

[['tiger', 'panthera', 'tigris', 'largest', 'extant', 'cat', 'species', 'member', 'genus', 'recognisable', 'dark', 'vertical', 'stripes', 'orange', 'brown', 'fur', 'lighter', 'underside', 'apex', 'predator', 'primarily', 'preying', 'ungulates', 'deer', 'wild', 'boar', 'territorial', 'generally', 'solitary', 'social', 'requiring', 'large', 'contiguous', 'areas', 'habitat', 'support', 'requirements', 'prey', 'rearing', 'offspring', 'cubs', 'stay', 'mother', 'two', 'years', 'become', 'independent', 'leave', 'home', 'range', 'establish', 'ranged', 'widely', 'eastern', 'anatolia', 'region', 'west', 'amur', 'river', 'basin', 'south', 'foothills', 'himalayas', 'bali', 'sunda', 'islands', 'since', 'early', 'century', 'populations', 'lost', 'least', 'historic', 'extirpated', 'western', 'central', 'asia', 'java', 'southeast', 'china', 'today', 'fragmented', 'stretching', 'siberian', 'temperate', 'forests', 'subtropical', 'tropical', 'indian', 'subcontinent', 'sumatra', 'listed', 'endangered', 'i

In [ ]:
documents = [TaggedDocument(doc, [df['Category'][i]]) for i, doc in enumerate(tokens)]
print(documents[0])

TaggedDocument(['tiger', 'panthera', 'tigris', 'largest', 'extant', 'cat', 'species', 'member', 'genus', 'recognisable', 'dark', 'vertical', 'stripes', 'orange', 'brown', 'fur', 'lighter', 'underside', 'apex', 'predator', 'primarily', 'preying', 'ungulates', 'deer', 'wild', 'boar', 'territorial', 'generally', 'solitary', 'social', 'requiring', 'large', 'contiguous', 'areas', 'habitat', 'support', 'requirements', 'prey', 'rearing', 'offspring', 'cubs', 'stay', 'mother', 'two', 'years', 'become', 'independent', 'leave', 'home', 'range', 'establish', 'ranged', 'widely', 'eastern', 'anatolia', 'region', 'west', 'amur', 'river', 'basin', 'south', 'foothills', 'himalayas', 'bali', 'sunda', 'islands', 'since', 'early', 'century', 'populations', 'lost', 'least', 'historic', 'extirpated', 'western', 'central', 'asia', 'java', 'southeast', 'china', 'today', 'fragmented', 'stretching', 'siberian', 'temperate', 'forests', 'subtropical', 'tropical', 'indian', 'subcontinent', 'sumatra', 'listed', 'e

In [ ]:
all = documents

In [ ]:
train_tagged, test_tagged = train_test_split(all, test_size=0.3, random_state=42)

In [ ]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [ ]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [ ]:
model1 = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model2 = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model3 = Doc2Vec(dm=1, dm_mean=1, vector_size=300, window=10, negative=5, min_count=1, workers=5, alpha=0.065, min_alpha=0.065)

In [ ]:
model1.build_vocab(train_tagged)
model2.build_vocab(train_tagged)
model3.build_vocab(train_tagged)

In [ ]:
%%time
for epoch in range(30):
    model1.train(utils.shuffle([x for x in tqdm(train_tagged)]), total_examples=len(train_tagged), epochs=1)
    model1.alpha -= 0.002
    model1.min_alpha = model1.alpha

100%|██████████| 52/52 [00:00<00:00, 173926.48it/s]


CPU times: user 553 ms, sys: 40.8 ms, total: 594 ms
Wall time: 736 ms


In [ ]:
filename = '/content/drive/MyDrive/d2v_model1i.pkl'
pickle.dump(model1, open(filename, 'wb'))

In [ ]:
y_train, X_train = vec_for_learning(model1, train_tagged)
y_test, X_test = vec_for_learning(model1, test_tagged)
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
y_pred_tr = logreg.predict(X_train)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print('Train accuracy %s' % accuracy_score(y_train, y_pred_tr))
print('Train F1 score: {}'.format(f1_score(y_train, y_pred_tr, average='weighted')))

Testing accuracy 0.5217391304347826
Testing F1 score: 0.5386128364389233
Train accuracy 0.7692307692307693
Train F1 score: 0.7692307692307693


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
filename = '/content/drive/MyDrive/best1.pkl'
pickle.dump(logreg, open(filename, 'wb'))

In [ ]:
for epoch in range(30):
    model2.train(utils.shuffle([x for x in tqdm(train_tagged)]), total_examples=len(train_tagged), epochs=1)
    model2.alpha -= 0.002
    model2.min_alpha = model2.alpha

100%|██████████| 52/52 [00:00<00:00, 207323.01it/s]


In [ ]:
#najlepszy model doc2vec
filename = '/content/drive/MyDrive/d2v_model22i.pkl'
pickle.dump(model2, open(filename, 'wb'))

In [ ]:
y_train, X_train = vec_for_learning(model2, train_tagged)
y_test, X_test = vec_for_learning(model2, test_tagged)
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
y_pred_tr = logreg.predict(X_train)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print('Train accuracy %s' % accuracy_score(y_train, y_pred_tr))
print('Train F1 score: {}'.format(f1_score(y_train, y_pred_tr, average='weighted')))

Testing accuracy 0.9565217391304348
Testing F1 score: 0.9569609134826527
Train accuracy 1.0
Train F1 score: 1.0


In [ ]:
#najlepszy model regresja
filename = '/content/drive/MyDrive/best101.pkl'
pickle.dump(logreg, open(filename, 'wb'))

In [ ]:
for epoch in range(30):
    model3.train(utils.shuffle([x for x in tqdm(train_tagged)]), total_examples=len(train_tagged), epochs=1)
    model3.alpha -= 0.002
    model3.min_alpha = model3.alpha

100%|██████████| 52/52 [00:00<00:00, 15507.95it/s]


In [ ]:
filename = '/content/drive/MyDrive/d2v_model3i.pkl'
pickle.dump(model3, open(filename, 'wb'))

In [ ]:
y_train, X_train = vec_for_learning(model3, train_tagged)
y_test, X_test = vec_for_learning(model3, test_tagged)
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
y_pred_tr = logreg.predict(X_train)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print('Train accuracy %s' % accuracy_score(y_train, y_pred_tr))
print('Train F1 score: {}'.format(f1_score(y_train, y_pred_tr, average='weighted')))

Testing accuracy 0.8695652173913043
Testing F1 score: 0.8766798418972332
Train accuracy 1.0
Train F1 score: 1.0


In [ ]:
filename = '/content/drive/MyDrive/best3.pkl'
pickle.dump(logreg, open(filename, 'wb'))

Poniżej testowanie wyników modelu.

In [ ]:
#class DataProcessor:
#  def __init__(self):
#    self.tokenizer = RegexpTokenizer(r'\w+')
#  def preprocess_text(self, text):
#    text = text.lower()
#    tokens = self.tokenizer.tokenize(text)
#    tokens = [word.lower() for word in tokens]
#    tokens = [word for word in tokens if word.isalpha()]
#    return ' '.join(tokens)
#  def preprocess_dataframe(self, dataframe, in_column = 'RawWiki', out_column = 'CleanWiki'):
#    dataframe.loc[:, out_column] = dataframe.loc[:, in_column].apply(self.preprocess_text)

In [ ]:
#from nltk.corpus import stopwords
#from nltk.tokenize import RegexpTokenizer
#doc2vec = pickle.load(open('/content/drive/MyDrive/d2v_model22i.pkl', 'rb'))
#logreg = pickle.load(open('/content/drive/MyDrive/best101.pkl', 'rb'))
#text = "The tiger (Panthera tigris) is the largest extant cat species and a member of the genus Panthera. It is most recognisable for its dark vertical stripes on orange-brown fur with a lighter underside. It is an apex predator, primarily preying on ungulates such as deer and wild boar. It is territorial and generally a solitary but social predator, requiring large contiguous areas of habitat, which support its requirements for prey and rearing of its offspring. Tiger cubs stay with their mother for about two years, before they become independent and leave their mother's home range to establish their own. The tiger once ranged widely from the Eastern Anatolia Region in the west to the Amur River basin, and in the south from the foothills of the Himalayas to Bali in the Sunda islands. Since the early 20th century, tiger populations have lost at least 93% of their historic range and have been extirpated in Western and Central Asia, from the islands of Java and Bali, and in large areas of Southeast and South Asia and China. Today's tiger range is fragmented, stretching from Siberian temperate forests to subtropical and tropical forests on the Indian subcontinent and Sumatra.  The tiger is listed as endangered on the IUCN Red List. As of 2015, the global wild tiger population was estimated to number between 3,062 and 3,948 mature individuals, with most of the populations living in small pockets isolated from each other. India currently hosts the largest tiger population. Major reasons for population decline are habitat destruction, habitat fragmentation and poaching. Tigers are also victims of human–wildlife conflict, in particular in range countries with a high human population density. The tiger is among the most recognisable and popular of the world's charismatic megafauna. It featured prominently in ancient mythology and folklore and continues to be depicted in modern films and literature, appearing on many flags, coats of arms and as mascots for sporting teams. The tiger is the national animal of India, Bangladesh, Malaysia and South Korea."
#clean_text = DataProcessor().preprocess_text(text)
#def getVector(doc2vec, text):
#  words = text.split()
#  vector = doc2vec.infer_vector(words, steps=20)
#  return vector
#def predict(logreg, vector):
#  return logreg.predict(vector.reshape(1, -1))[0]
#v = getVector(doc2vec, clean_text)
#o = predict(logreg, v)
#print(o)
#Should be Animals

Animals
